## 녹내장테스트

In [5]:
userID = '000000001'

import cv2
from cvzone.FaceMeshModule import FaceMeshDetector
import mediapipe as mp
import numpy as np
from PIL import ImageFont, ImageDraw, Image
import pandas as pd
import eyeTest as et
import datetime
import time

d_start = 30    # 시작거리
d_end = 50      # 끝나는 거리
disease_name = '녹내장'

now = datetime.datetime.now()
nowDatetime = now.strftime('%Y-%m-%d %H:%M:%S')

detector = FaceMeshDetector(maxFaces=1)
mp_hands = mp.solutions.hands

width = 1260
height = 720
testEnd = False
another = False
counter = 0
counterPause = 0
selectionSpeed = 8
btn_size = 40
List = [] 
eye = '오른쪽눈'

font = ImageFont.truetype('../static/fonts/H2GSRB.TTF', 40)
true = cv2.resize(cv2.imread('../static/button/Yes.png', cv2.IMREAD_UNCHANGED),(80,80))
false = cv2.resize(cv2.imread('../static/button/No.png', cv2.IMREAD_UNCHANGED),(80,80))
logo = cv2.resize(cv2.imread('../static/button/eye.png', cv2.IMREAD_UNCHANGED),(80,80))
background = cv2.resize(cv2.imread('../static/image/background.jpg'),(1000,630))
disease = cv2.resize(cv2.imread('../static/image/noknaezang/glaucoma.jpg'),(740,440))
test = cv2.resize(cv2.imread('../static/image/test.png', cv2.IMREAD_UNCHANGED),(300,210))

def text_def(xy,text_name,fontstyle,text_color):
    global image
    image = Image.fromarray(image)    
    draw = ImageDraw.Draw(image)    
    draw.text(xy=xy,  text=text_name, font=fontstyle, fill= text_color)
    image = np.array(image) 

cap = cv2.VideoCapture(0)

with mp_hands.Hands(max_num_hands=2,min_detection_confidence=0.5,min_tracking_confidence=0.5) as hands:
    while True:        
        if cap.get(cv2.CAP_PROP_POS_FRAMES) == cap.get(cv2.CAP_PROP_FRAME_COUNT):
            cap.set(cv2.CAP_PROP_POS_FRAMES, 0)
        cap.set(cv2.CAP_PROP_FRAME_WIDTH, width)
        cap.set(cv2.CAP_PROP_FRAME_HEIGHT, height)
        success, image = cap.read()
        
        image = cv2.flip(image, 1)
        image, faces = detector.findFaceMesh(image, draw=False)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = hands.process(image)
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        et.img_size(image,disease,630,360)

        if testEnd is False: 
            text_def((100, 35),  f'얼구정면인 상태에서 그림같은 자세를 취하고 오른쪽/왼쪽 \n손가락이 보이면 V, 안보이면 X에 손가락을 대주세요.', font, (0, 0, 0))
            if another is False:
                if results.multi_hand_landmarks:                   
                    for hand_landmarks in results.multi_hand_landmarks:
                        finger = hand_landmarks.landmark[8]
                        h, w, _=image.shape 
                        finger=(int(finger.x*w), int(finger.y*h))   
                        if (abs(finger[0] - 260)<btn_size) & (abs(finger[1] - 360) < btn_size):
                            counter += 1
                            cv2.ellipse(image, (260,360), (btn_size, btn_size), 0, 0,counter*selectionSpeed, (255, 0, 255), 20)
                            if counter * selectionSpeed > 360:
                                counter = 0
                                answer = '정상'
                                List.append({
                                    'ID':userID,
                                    '시간':nowDatetime,
                                    '눈':eye,
                                    '여부':answer
                                    })
                                df = pd.DataFrame(List)
                                df.to_csv(f'../static/csv파일/{disease_name}.csv')      
                                print(f'{answer}으로 의심됩니다.')
                                timeStart = time.time()
                                another = True
                                
                        elif (abs(finger[0] - 1000)<btn_size) & (abs(finger[1] - 360) < btn_size):
                            counter += 1
                            cv2.ellipse(image, (1000,360), (btn_size, btn_size), 0, 0,counter*selectionSpeed, (255, 0, 255), 20)
                            if counter * selectionSpeed > 360:
                                counter = 0
                                answer = disease_name
                                List.append({
                                    'ID':userID,
                                    '시간':nowDatetime,
                                    '눈':eye,
                                    '여부':answer
                                    })
                                df = pd.DataFrame(List)
                                df.to_csv(f'../static/csv파일/{disease_name}.csv')       
                                print(f'{answer}입니다.')
                                timeStart = time.time()
                                another = True
                        else:
                            pass

                        cv2.circle(image,finger,20,(255,0,0),2,cv2.LINE_AA) #파란색             

                if len(List) == 2 :   
                    timeStart = time.time() 
                    testEnd = True                   

                et.overlay(image, *(260,360), 40, 40, true)   
                et.overlay(image, *(1000,360), 40, 40, false)    
                et.overlay(image,*(150,630),150,120,test)   # 횟수창
                text_def((65, 607),f'{eye}',ImageFont.truetype('../static/fonts/H2GSRB.TTF', 30),(0,0,0))

            else:
                et.img_size(image,background,630,360)
                eye = '왼쪽눈'
                text_def((370, 250),'테스트 계속하기',ImageFont.truetype('../static/fonts/H2GSRB.TTF', 70),(0, 0, 0)) 
                text_def((350, 380),f'{int(6-(time.time()-timeStart))}초후 {eye} 테스트 시작합니다.',ImageFont.truetype('../static/fonts/H2GSRB.TTF', 40),(0, 0, 0))
                if int(6-(time.time()-timeStart)) == 0: 
                    another = False
        else:
            et.img_size(image,background,630,360)
            text_def((300, 220),  f"{disease_name}테스트 검사결과 ", ImageFont.truetype('../static/fonts/H2GSRB.TTF', 60),  (0, 0, 0))
            text_def((480, 360),  f"{List[0]['눈']}: {List[0]['여부']} ", font,  (0, 0, 0))
            text_def((480, 450),  f"{List[1]['눈']}: {List[1]['여부']} ", font,  (0, 0, 0))
            text_def((410, 580),f'{int(11-(time.time()-timeStart))} 초후 테스트 종료합니다.',ImageFont.truetype('../static/fonts/H2GSRB.TTF', 40),(0, 0, 0))
            if int(11-(time.time()-timeStart)) == 0: 
                break

        et.overlay(image, *(50,50), 40, 40, logo) 

        cv2.imshow("Image", image)
        key = cv2.waitKey(1)
        if key == ord('q'):
            break
print(List)
print('저장 완료됬습니다.')

cv2.destroyAllWindows()
cap.release()


정상으로 의심됩니다.
녹내장입니다.
[{'ID': '000000001', '시간': '2023-02-28 13:31:01', '눈': '오른쪽눈', '여부': '정상'}, {'ID': '000000001', '시간': '2023-02-28 13:31:01', '눈': '왼쪽눈', '여부': '녹내장'}]
저장 완료됬습니다.
